In [1]:
import requests
import re
import json
import os
import pandas as pd
import sys
import numpy as np

# project lib
PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA-UFO')
sys.path.append(PROJECT_SRC_PATH)
import dataset

In [2]:
# Set a seed for reproducibility
np.random.seed(42)  # You can choose any number as your seed

# Define paths for data
path_data_NLD = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-NLD.pkl')
path_data_FRA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-FRA.pkl')
path_data_ESP = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-ESP.pkl')
processed_df_NLD = "/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD_adv.csv"
processed_df_ESP = "/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv"
processed_df_FRA = "/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA_adv.csv"
processed_df_ALL = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Image directory paths
image_dir_map = {
    'NLD': '/workspace/workspace/ufo-prediction/image_data_NLD_adv',
    'FRA': '/workspace/workspace/ufo-prediction/image_data_FRA_adv',
    'ESP': '/workspace/workspace/ufo-prediction/image_data_ESP_adv',
    'ALL': '/workspace/workspace/ufo-prediction/image_data'
}

# Ask for user input
country_code = input("Enter country code (FRA, NLD, ESP) or ALL: ").upper()

# Map user input to the correct path
path_data_map = {
    'NLD': processed_df_NLD,
    'FRA': processed_df_FRA,
    'ESP': processed_df_ESP,
    'ALL': processed_df_ALL
}

raw_data_path_map = {
'NLD': path_data_NLD,
'FRA': path_data_FRA,
'ESP': path_data_ESP
}
# Check if the input is valid
if country_code not in path_data_map:
    print("Invalid country code or specification. Please enter FRA, NLD, ESP, or ALL.")
else:
    processed_df_path = path_data_map[country_code]
    # Set directory based on country code
    current_directory = image_dir_map[country_code]

    # Check if the new directory exists, if not, create it
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
        print(f"Directory {current_directory} created.")
    else:
        print(f"Directory {current_directory} already exists.")

    # Process for ALL
    if country_code == 'ALL':
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print("Creating a new processed DataFrame for ALL")
            path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
            df = pd.read_csv(path_data_RCA, encoding='latin1')
            kartaview_keys = df[['lon', 'lat','age_right', 'id', 'PropertyKey_ID']]
            kartaview_keys.to_csv(processed_df_path, index=False)
            

    # Process for FRA, NLD, ESP
    else:
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print(f"Creating a new processed DataFrame for {country_code}")
            df_path = raw_data_path_map[country_code]
            df = pd.read_pickle(df_path)
            print("Loaded DataFrame from", df_path)
            sampled_df = df.sample(n=1500000, random_state=42)
            kartaview_keys = sampled_df[['lon', 'lat', 'age', 'id']].rename(columns={'age': 'age_right'})
            kartaview_keys.to_csv(processed_df_path, index=False)

print(kartaview_keys) 

Enter country code (FRA, NLD, ESP) or ALL: ESP
Directory /workspace/workspace/ufo-prediction/image_data_ESP_adv already exists.
Loaded processed DataFrame from /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
             lon        lat  age_right                         id
0      -1.798480  40.972110     1940.0    v0.1-ESP.4.4.1.53_1-112
1      -4.128549  37.172721     1960.0   v0.1-ESP.1.4.5.29_1-9551
2       2.608557  39.594611     2000.0  v0.1-ESP.13.1.8.1_1-40343
3      -2.020573  40.296965     1900.0    v0.1-ESP.4.3.2.12_1-371
4       2.496946  39.511195     2002.0  v0.1-ESP.13.1.7.4_1-12584
...          ...        ...        ...                        ...
894009 -5.984130  37.675687     2011.0    v0.1-ESP.1.8.1.4_1-5558
894010  2.505601  41.879092     1900.0    v0.1-ESP.6.2.6.19_1-823
894011 -3.924419  37.463344     1980.0   v0.1-ESP.1.6.9.1_1-10785
894012 -0.220692  38.513329     1945.0  v0.1-ESP.10.1.7.17_1-5766
894013 -4.000000        NaN        NaN         

In [3]:
print(current_directory)
print(kartaview_keys)

/workspace/workspace/ufo-prediction/image_data_ESP_adv
             lon        lat  age_right                         id
0      -1.798480  40.972110     1940.0    v0.1-ESP.4.4.1.53_1-112
1      -4.128549  37.172721     1960.0   v0.1-ESP.1.4.5.29_1-9551
2       2.608557  39.594611     2000.0  v0.1-ESP.13.1.8.1_1-40343
3      -2.020573  40.296965     1900.0    v0.1-ESP.4.3.2.12_1-371
4       2.496946  39.511195     2002.0  v0.1-ESP.13.1.7.4_1-12584
...          ...        ...        ...                        ...
894009 -5.984130  37.675687     2011.0    v0.1-ESP.1.8.1.4_1-5558
894010  2.505601  41.879092     1900.0    v0.1-ESP.6.2.6.19_1-823
894011 -3.924419  37.463344     1980.0   v0.1-ESP.1.6.9.1_1-10785
894012 -0.220692  38.513329     1945.0  v0.1-ESP.10.1.7.17_1-5766
894013 -4.000000        NaN        NaN                        NaN

[894014 rows x 4 columns]


In [ ]:
# Assuming current_directory and processed_df_path are set from the previous code chunk
print(f"Images will be saved in: {current_directory}")
print(f"Using kartaview_keys from: {processed_df_path}")

# Consistency check between image directory and kartaview_keys path
expected_csv_map = {
    '/workspace/workspace/ufo-prediction/image_data': '/workspace/workspace/ufo-prediction/demo/kartaview_key.csv',
    '/workspace/workspace/ufo-prediction/image_data_NLD': '/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv',
    '/workspace/workspace/ufo-prediction/image_data_FRA': '/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv',
    '/workspace/workspace/ufo-prediction/image_data_ESP': '/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv',
}

# Stop the code if using image_data directory
if current_directory == '/workspace/workspace/ufo-prediction/image_data':
    print("Download for the 'image_data' directory has already been completed. Stopping execution.")
    # Use `exit()` or `sys.exit()` depending on your environment
    exit()

if processed_df_path != expected_csv_map.get(current_directory):
    print("Inconsistency detected between the image directory and the kartaview_keys path. Please check.")
    exit()

image_count = {} 
print("Number of buildings remaining: ",len(kartaview_keys))

#Initialise a counter for the loop iterations
iteration_counter = 0

for index, row in kartaview_keys.iterrows():
    iteration_counter += 1 # Increment the counter with each iteration
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                # Initialize or update the image count for the current ID
                building_id = row['id']  # Assuming 'id' column exists in your DataFrame
                if building_id not in image_count:
                    image_count[building_id] = 0

                for image_url in filtered_urls:
                    image_count[building_id] += 1  # Increment the image count for the building
                    subscript = image_count[building_id]  # Subscript for the file name
                    file_name = f"{row['age_right']}_{building_id}_{subscript}.jpg"
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Only save the updated DataFrame to a CSV file every 100th instance
    if iteration_counter % 100 == 0:
        print(f"Saving progress at iteration {iteration_counter} to {processed_df_path}. ")
        print("Number of buildings remaining: ",len(kartaview_keys))
        kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))
        
if iteration_counter % 100 != 0:
    print(f"Saving final progress")
    print("Number of buildings remaining: ",len(kartaview_keys))
    kartaview_keys.to_csv(processed_df_path, index=False)

Images will be saved in: /workspace/workspace/ufo-prediction/image_data_ESP_adv
Using kartaview_keys from: /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
Inconsistency detected between the image directory and the kartaview_keys path. Please check.
Number of buildings remaining:  894014
No suitable images found for location: lon=-1.798480, lat=40.972110
No suitable images found for location: lon=-1.79848, lat=40.97211
No suitable images found for location: lon=-1.7985, lat=40.9721
No suitable images found for location: lon=-1.798, lat=40.972
Unable to retrieve data from the API with sufficient precision for location: lon=-1.798480170184035, lat=40.97210984198054
No suitable images found for location: lon=-4.128549, lat=37.172721
No suitable images found for location: lon=-4.12855, lat=37.17272
No suitable images found for location: lon=-4.1285, lat=37.1727
No suitable images found for location: lon=-4.129, lat=37.173
Unable to retrieve data from the API with sufficie

No suitable images found for location: lon=-2.241320, lat=40.450428
No suitable images found for location: lon=-2.24132, lat=40.45043
No suitable images found for location: lon=-2.2413, lat=40.4504
No suitable images found for location: lon=-2.241, lat=40.450
Unable to retrieve data from the API with sufficient precision for location: lon=-2.241320219983023, lat=40.45042794222531
No suitable images found for location: lon=-5.987302, lat=37.675862
No suitable images found for location: lon=-5.98730, lat=37.67586
No suitable images found for location: lon=-5.9873, lat=37.6759
No suitable images found for location: lon=-5.987, lat=37.676
Unable to retrieve data from the API with sufficient precision for location: lon=-5.9873019711345545, lat=37.67586184863664
No suitable images found for location: lon=-1.269283, lat=38.074066
No suitable images found for location: lon=-1.26928, lat=38.07407
No suitable images found for location: lon=-1.2693, lat=38.0741
No suitable images found for locati

No suitable images found for location: lon=2.064, lat=41.497
Unable to retrieve data from the API with sufficient precision for location: lon=2.0642051191492072, lat=41.496637274673006
No suitable images found for location: lon=-1.860682, lat=38.989526
No suitable images found for location: lon=-1.86068, lat=38.98953
No suitable images found for location: lon=-1.8607, lat=38.9895
No suitable images found for location: lon=-1.861, lat=38.990
Unable to retrieve data from the API with sufficient precision for location: lon=-1.8606817559024893, lat=38.98952640842225
No suitable images found for location: lon=-0.442913, lat=38.710331
No suitable images found for location: lon=-0.44291, lat=38.71033
No suitable images found for location: lon=-0.4429, lat=38.7103
No suitable images found for location: lon=-0.443, lat=38.710
Unable to retrieve data from the API with sufficient precision for location: lon=-0.4429134852676199, lat=38.71033110227857
No suitable images found for location: lon=3.07

No suitable images found for location: lon=-8.684471, lat=42.221211
No suitable images found for location: lon=-8.68447, lat=42.22121
No suitable images found for location: lon=-8.6845, lat=42.2212
No suitable images found for location: lon=-8.684, lat=42.221
Unable to retrieve data from the API with sufficient precision for location: lon=-8.684471232862949, lat=42.22121057253614
No suitable images found for location: lon=-4.005886, lat=41.153093
No suitable images found for location: lon=-4.00589, lat=41.15309
No suitable images found for location: lon=-4.0059, lat=41.1531
No suitable images found for location: lon=-4.006, lat=41.153
Unable to retrieve data from the API with sufficient precision for location: lon=-4.005885595527532, lat=41.15309262077249
No suitable images found for location: lon=-0.384583, lat=39.732951
No suitable images found for location: lon=-0.38458, lat=39.73295
No suitable images found for location: lon=-0.3846, lat=39.7330
No suitable images found for locatio

No suitable images found for location: lon=-5.357, lat=40.152
Unable to retrieve data from the API with sufficient precision for location: lon=-5.357134700660822, lat=40.15151774914527
No suitable images found for location: lon=-4.643802, lat=39.716061
No suitable images found for location: lon=-4.64380, lat=39.71606
No suitable images found for location: lon=-4.6438, lat=39.7161
No suitable images found for location: lon=-4.644, lat=39.716
Unable to retrieve data from the API with sufficient precision for location: lon=-4.643802457492077, lat=39.71606146606128
No suitable images found for location: lon=-9.032293, lat=42.667697
No suitable images found for location: lon=-9.03229, lat=42.66770
No suitable images found for location: lon=-9.0323, lat=42.6677
No suitable images found for location: lon=-9.032, lat=42.668
Unable to retrieve data from the API with sufficient precision for location: lon=-9.03229307248667, lat=42.66769702622808
No suitable images found for location: lon=-4.3953

...

No suitable images found for location: lon=3.064, lat=39.883
Unable to retrieve data from the API with sufficient precision for location: lon=3.0640437564142253, lat=39.88278100606632
No suitable images found for location: lon=-3.711803, lat=39.409970
No suitable images found for location: lon=-3.71180, lat=39.40997
No suitable images found for location: lon=-3.7118, lat=39.4100
No suitable images found for location: lon=-3.712, lat=39.410
Unable to retrieve data from the API with sufficient precision for location: lon=-3.711803380377327, lat=39.409970432713685
No suitable images found for location: lon=-4.720910, lat=36.493762
No suitable images found for location: lon=-4.72091, lat=36.49376
No suitable images found for location: lon=-4.7209, lat=36.4938
No suitable images found for location: lon=-4.721, lat=36.494
Unable to retrieve data from the API with sufficient precision for location: lon=-4.720909944558955, lat=36.4937617481044
No suitable images found for location: lon=-0.2443

No suitable images found for location: lon=-4.034984, lat=39.870073
No suitable images found for location: lon=-4.03498, lat=39.87007
No suitable images found for location: lon=-4.0350, lat=39.8701
No suitable images found for location: lon=-4.035, lat=39.870
Unable to retrieve data from the API with sufficient precision for location: lon=-4.034983746794839, lat=39.87007306810255
No suitable images found for location: lon=-2.427491, lat=39.406569
No suitable images found for location: lon=-2.42749, lat=39.40657
No suitable images found for location: lon=-2.4275, lat=39.4066
No suitable images found for location: lon=-2.427, lat=39.407
Unable to retrieve data from the API with sufficient precision for location: lon=-2.4274906311460214, lat=39.40656904349695
No suitable images found for location: lon=-13.647416, lat=28.965620
No suitable images found for location: lon=-13.64742, lat=28.96562
No suitable images found for location: lon=-13.6474, lat=28.9656
No suitable images found for loc

No suitable images found for location: lon=-7.751772, lat=43.164234
No suitable images found for location: lon=-7.75177, lat=43.16423
No suitable images found for location: lon=-7.7518, lat=43.1642
No suitable images found for location: lon=-7.752, lat=43.164
Unable to retrieve data from the API with sufficient precision for location: lon=-7.751772179598687, lat=43.16423356753328
No suitable images found for location: lon=-1.696768, lat=38.591894
No suitable images found for location: lon=-1.69677, lat=38.59189
No suitable images found for location: lon=-1.6968, lat=38.5919
No suitable images found for location: lon=-1.697, lat=38.592
Unable to retrieve data from the API with sufficient precision for location: lon=-1.696767723011993, lat=38.59189447089505
No suitable images found for location: lon=-2.604035, lat=37.017906
No suitable images found for location: lon=-2.60403, lat=37.01791
No suitable images found for location: lon=-2.6040, lat=37.0179
No suitable images found for locatio

No suitable images found for location: lon=-3.615, lat=39.864
Unable to retrieve data from the API with sufficient precision for location: lon=-3.6145114377438863, lat=39.86448804665037
No suitable images found for location: lon=-0.178916, lat=38.972447
No suitable images found for location: lon=-0.17892, lat=38.97245
No suitable images found for location: lon=-0.1789, lat=38.9724
No suitable images found for location: lon=-0.179, lat=38.972
Unable to retrieve data from the API with sufficient precision for location: lon=-0.1789164271925759, lat=38.97244712674496
No suitable images found for location: lon=3.146761, lat=39.465763
No suitable images found for location: lon=3.14676, lat=39.46576
No suitable images found for location: lon=3.1468, lat=39.4658
No suitable images found for location: lon=3.147, lat=39.466
Unable to retrieve data from the API with sufficient precision for location: lon=3.1467605886338705, lat=39.46576303637882
No suitable images found for location: lon=-4.00825

No suitable images found for location: lon=-3.368461, lat=38.017759
No suitable images found for location: lon=-3.36846, lat=38.01776
No suitable images found for location: lon=-3.3685, lat=38.0178
No suitable images found for location: lon=-3.368, lat=38.018
Unable to retrieve data from the API with sufficient precision for location: lon=-3.3684610930648264, lat=38.01775886586179
No suitable images found for location: lon=-0.471977, lat=39.566476
No suitable images found for location: lon=-0.47198, lat=39.56648
No suitable images found for location: lon=-0.4720, lat=39.5665
No suitable images found for location: lon=-0.472, lat=39.566
Unable to retrieve data from the API with sufficient precision for location: lon=-0.4719766749040556, lat=39.566475745380544
No suitable images found for location: lon=-4.840650, lat=41.699169
No suitable images found for location: lon=-4.84065, lat=41.69917
No suitable images found for location: lon=-4.8406, lat=41.6992
No suitable images found for loca